# Enter an Activity Id and See a Summary of that Activity

In [5]:
from IPython.display import display, Markdown
import snakemd

import fitfile
from garmindb import ConfigManager
from garmindb.garmindb import GarminDb, Attributes, ActivitiesDb, Activities, ActivityLaps
from idbutils import Location

activity_id = input('Enter the id of the activity you would like to display')

doc = snakemd.new_doc("activity")

db_params_dict = ConfigManager.get_db_params()
garmin_db = GarminDb(db_params_dict)
garmin_act_db = ActivitiesDb(db_params_dict)
measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = unit_strings[fitfile.units.UnitTypes.distance_long]
altitude_units = unit_strings[fitfile.units.UnitTypes.altitude]
temp_units = unit_strings[fitfile.units.UnitTypes.tempurature]

activity = Activities.get(garmin_act_db, activity_id)

doc.add_header(activity.name)
doc.add_paragraph(f'Id: {activity_id} Sport: {activity.sport} Activity type: {activity.type}')
if activity.description:
    doc.add_paragraph(activity.description)

doc.add_header("Summary", 3)
start_loc = Location(activity.start_lat , activity.start_long)
end_loc = Location(activity.stop_lat , activity.stop_long)
doc.add_table([f'Distance ({distance_units})', "Ellapsed Time", "Moving Time", "Calories", f"Average Tempature (degrees {temp_units})", 'Start Location', 'End Location'],
              [[activity.distance, activity.elapsed_time, activity.moving_time, activity.calories, activity.avg_temperature,
               snakemd.InlineText(str(start_loc), start_loc.to_google_maps_url()), snakemd.InlineText(str(end_loc), end_loc.to_google_maps_url())]])

doc.add_header("Laps", 3)
laps = ActivityLaps.get_activity(garmin_act_db, activity_id)
lap_stats = []
for lap in laps:
    start_loc = Location(lap.start_lat , lap.start_long)
    end_loc = Location(lap.stop_lat , lap.stop_long)
    lap_stats.append(
        [lap.lap, lap.distance, lap.elapsed_time, lap.moving_time, lap.calories, lap.avg_temperature,
            snakemd.InlineText(str(start_loc), start_loc.to_google_maps_url()), snakemd.InlineText(str(end_loc), end_loc.to_google_maps_url())]
    )
doc.add_table(['Lap', f'Distance ({distance_units})', "Ellapsed Time", "Moving Time", "Calories", f"Average Tempature (degrees {temp_units})", 'Start Location', 'End Location'], lap_stats)

doc.add_header("Altitude", 3)
doc.add_table([f'Gained ({altitude_units})', f'Lost ({altitude_units})'], [[activity.ascent, activity.descent]])

doc.add_header("Heart Rate", 3)
doc.add_table(['Average', 'Maximum'], [[activity.avg_hr, activity.max_hr]])

doc.add_header("Respiration Rate", 3)
doc.add_table(['Average', 'Maximum'], [[activity.avg_rr, activity.max_rr]])

doc.add_header("Time In Heart Rate Zones", 3)
doc.add_table(
    ['Zone', 'HR Threshold', "Time"],
    [
        ["1", activity.hrz_1_hr, activity.hrz_1_time],
        ["2", activity.hrz_2_hr, activity.hrz_2_time],
        ["3", activity.hrz_3_hr, activity.hrz_3_time],
        ["4", activity.hrz_4_hr, activity.hrz_4_time],
        ["5", activity.hrz_5_hr, activity.hrz_5_time],
    ])

doc.add_header("Training Effect", 3)
doc.add_table(
    ['Type', 'Rating'],
    [
        ["Aerobic", activity.training_effect],
        ["Anaerobic", activity.anaerobic_training_effect],
    ])

display(Markdown(str(doc)))

# Cromesett Walk

Id: 8090181358 Sport: walking Activity type: recreation

Walk to conservation land with Michele and Maeve.

### Summary

| Distance (miles) | Ellapsed Time   | Moving Time     | Calories | Average Tempature (degrees F) | Start Location                                                                                                       | End Location                                                                                                         |
| ---------------- | --------------- | --------------- | -------- | ----------------------------- | -------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------- |
| 4.783818808272   | 01:45:14.524000 | 01:45:14.524000 | 453      | 68.0                          | [41.73853048123419, -70.73556767776608](http://maps.google.com/?ie=UTF8&q=41.73853048123419,-70.73556767776608&z=13) | [41.73597626388073, -70.73246545158327](http://maps.google.com/?ie=UTF8&q=41.73597626388073,-70.73246545158327&z=13) |

### Laps

| Lap | Distance (miles) | Ellapsed Time   | Moving Time     | Calories | Average Tempature (degrees F) | Start Location                                                                                                       | End Location                                                                                                         |
| --- | ---------------- | --------------- | --------------- | -------- | ----------------------------- | -------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------- |
| 0   | 0.999997527008   | 00:19:10.134000 | 00:19:10.134000 | 93       | 78.80000000000001             | [41.73853048123419, -70.73556767776608](http://maps.google.com/?ie=UTF8&q=41.73853048123419,-70.73556767776608&z=13) | [41.73597626388073, -70.73246545158327](http://maps.google.com/?ie=UTF8&q=41.73597626388073,-70.73246545158327&z=13) |
| 1   | 0.999997527008   | 00:23:36.203000 | 00:23:36.203000 | 96       | 69.80000000000001             | [41.73853048123419, -70.73556767776608](http://maps.google.com/?ie=UTF8&q=41.73853048123419,-70.73556767776608&z=13) | [41.73597626388073, -70.73246545158327](http://maps.google.com/?ie=UTF8&q=41.73597626388073,-70.73246545158327&z=13) |
| 2   | 0.999997527008   | 00:24:38.025000 | 00:24:38.025000 | 98       | 64.4                          | [41.73853048123419, -70.73556767776608](http://maps.google.com/?ie=UTF8&q=41.73853048123419,-70.73556767776608&z=13) | [41.73597626388073, -70.73246545158327](http://maps.google.com/?ie=UTF8&q=41.73597626388073,-70.73246545158327&z=13) |
| 3   | 0.999997527008   | 00:20:51.948000 | 00:20:51.948000 | 91       | 68.0                          | [41.73853048123419, -70.73556767776608](http://maps.google.com/?ie=UTF8&q=41.73853048123419,-70.73556767776608&z=13) | [41.73597626388073, -70.73246545158327](http://maps.google.com/?ie=UTF8&q=41.73597626388073,-70.73246545158327&z=13) |
| 4   | 0.78382870024    | 00:16:58.214000 | 00:16:58.214000 | 75       | 64.4                          | [41.73853048123419, -70.73556767776608](http://maps.google.com/?ie=UTF8&q=41.73853048123419,-70.73556767776608&z=13) | [41.73597626388073, -70.73246545158327](http://maps.google.com/?ie=UTF8&q=41.73597626388073,-70.73246545158327&z=13) |

### Altitude

| Gained (feet) | Lost (feet) |
| ------------- | ----------- |
| 134.5144359   | 751.3123371 |

### Heart Rate

| Average | Maximum |
| ------- | ------- |
| 109     | 132     |

### Respiration Rate

| Average | Maximum |
| ------- | ------- |
| None    | None    |

### Time In Heart Rate Zones

| Zone | HR Threshold | Time            |
| ---- | ------------ | --------------- |
| 1    | 118          | 00:19:12.003000 |
| 2    | 133          | 00:00:00        |
| 3    | 146          | 00:00:00        |
| 4    | 160          | 00:00:00        |
| 5    | 173          | 00:00:00        |

### Training Effect

| Type      | Rating |
| --------- | ------ |
| Aerobic   | 2.0    |
| Anaerobic | 0.0    |